In [83]:
import http.client, urllib.parse, json
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import html5lib
from util_html import *

In [62]:
def mediastack_query(keywords, languages, countries, date, offset, access_key = '0392ab082f336de31195dd7065260751', sort = 'published_desc'):
    conn = http.client.HTTPConnection('api.mediastack.com')
    params = ('access_key=' + str(access_key) + '&keywords=' + str(keywords) + '&sort='
              + str(sort) + '&languages=' + str(languages) + '&countries=' + str(countries) 
              + '&date=' + str(date) + '&offset=' + str(offset) + '&limit=100')
    conn.request('GET', '/v1/news?{}'.format(params))
    res = conn.getresponse()
    data = res.read()
    query_content=(data.decode('utf-8'))
    query = json.loads(query_content)
    #articles = query["data"]
    #return articles
    return query

In [8]:
def extract_metadata(article):
    # Extract the publication date
    published_at = article['published_at']
    if published_at:
        date, time = published_at.split("T")        
    else:
        date = ""
        time = ""

    # Extract meta data
    url = article ['url']
    title= article['title'] 
    
    # category associated with the given news article
    category = article['category']
    
    # country code associated with given article 
    country = article ['country']
    
    return date, time, title, url, category, country

In [81]:
articled = mediastack_query(keywords='abortion', languages='en', countries='us', date='2023-10-25',offset='0')

In [82]:
articled

{'pagination': {'limit': 100, 'offset': 0, 'count': 0, 'total': 0}, 'data': []}

In [25]:
def write_output(outfile_keywords, articles):

    outfile = "../results/mediastack_results/" + outfile_keywords + "_overview.tsv"


    with open(outfile, "w", encoding = "utf-8") as f:
        #date, time, title, article_url, category, country = extract_metadata(articles)

        f.write("Publication Date\tTime\tAuthor\tTitle\tURL\tText\n")
    
        for i, article in enumerate(articles):
        
        # Extract metadata
            date, time, title, article_url, category, country = extract_metadata(article)
        
        # Extract content
            article_content = url_to_html(article_url)
            author = parse_author(article_content)
            content = parse_news_text(article_content)
        
        # We remove the newlines from the content, so that we can easily store it in a single line. 
        # Keep in mind, that newlines can also carry meaning.
        # For example, they separate paragraphs and this information is lost in the analysis, if we remove them. 
            content = content.replace("\n", "")
        
        # We want the fields to be separated by tabulators (\t)
            output = "\t".join([date, time, author, title, article_url, content])
            f.write(output +"\n")

In [26]:
write_output(outfile_keywords = 'abortion', articles = articled)

In [41]:
df.drop_duplicates(subset='Time', inplace=True)
df

,Publication Date,Time,Author,Title,URL,Text
0,2023-11-02,19:33:44+00:00,NaN,Kefir Vs Kombucha: Which Fermented Drink Is Be...,https://www.healthcanal.com/nutrition/healthy-...,NaN
1,2023-11-02,16:44:01+00:00,NaN,Miyoko’s Production Moving Out of Petaluma,https://www.ksro.com/2023/11/02/miyokos-produc...,Miyoko’s Creamery is closing its production fa...
2,2023-11-02,08:31:10+00:00,Piri Medya A.Ş.,Veganlara özel Humm Organic lezzetleri,https://www.yenisafak.com/ekonomi/veganlara-oz...,Sürdürülebilir organik tarımı ve üretimi deste...
3,2023-11-01,22:18:39+00:00,https://www.hellomagazine.com/author/orin-carlin/,Everything you need to know about vegan beauty,https://www.hellomagazine.com/hfm/wish-list/50...,Vegan beauty has come on leaps and bounds in t...
4,2023-11-01,19:24:44+00:00,NaN,World Vegan Day: Is A Vegan Diet Wise?,https://euroweeklynews.com/2023/11/01/world-ve...,NaN
...,...,...,...,...,...,...
91,2023-10-03,08:26:27+00:00,https://www.theguardian.com/profile/amelia-tait,‘I can’t kill a wolf but will happily watch a ...,https://www.theguardian.com/games/2023/oct/03/...,From being unable to harvest ‘little sister’ c...
92,2023-10-03,08:04:42+00:00,Bridie Pearson-jones,Perfect for a posh Christmas! Ocado unveils it...,https://www.dailymail.co.uk/femail/food/articl...,"Published: 08:37 GMT, 3 October 2023 | Update..."
93,2023-10-02,15:09:05+00:00,NaN,The BMW i5 Is The Impossible Burger Of Electri...,https://jalopnik.com/the-bmw-i5-is-the-impossi...,There’s a common trope in automotive writing –...
94,2023-10-02,13:31:04+00:00,MarketScreener,Princess Cruises Introduces Expansive Vegan Me...,https://www.marketscreener.com/quote/stock/CAR...,"SANTA CLARITA, Calif., Oct. 2, 2023 /PRNewswir..."


In [48]:
def remove_duplicates(resultfile, cleaned_keyword):
    df = pd.read_csv(resultfile, delimiter="\t", error_bad_lines=False)
    df.drop_duplicates(subset='Time', inplace=True)
    cleaned_outfile = "../results/mediastack_results/" + cleaned_keyword + "_overview.csv"
    df.to_csv(cleaned_outfile, index=False)

In [49]:
remove_duplicates('../results/mediastack_results/veganism_overview.tsv','veganism')

/tmp/ipykernel_22702/167484004.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(resultfile, delimiter="\t", error_bad_lines=False)
Skipping line 9: expected 6 fields, saw 69
Skipping line 61: expected 6 fields, saw 192
Skipping line 76: expected 6 fields, saw 19
Skipping line 87: expected 6 fields, saw 69

